In [1]:
import json
import os
import requests
from pprint import pprint
import random
import datetime

In [2]:
s = requests.session()
s.cookies.clear()

In [3]:
OAUTH = ""

# Functions

Jump to:

1. [Update Layer](#layers)
1. [Update Dataset](#datasets)

In [4]:
def searchLayer(search, app='gfw'):
    search = search.strip().split(' ')
    
    hash = random.getrandbits(16)
    url = f'https://api.resourcewatch.org/v1/dataset?app={app}&includes=layer,vocabulary,metadata&page[size]=10000&hash={hash}'

    r = requests.get(url)
#     print(r.url)

    datasets = r.json().get('data', None)
    
    found = []
    for s in search:
        for d in datasets:
            ds_id = d.get('id')
            d_name = d.get('attributes').get('name').lower()
            slug = d.get('attributes').get('slug').lower()

            layers = d.get('attributes').get('layer', None)

            if layers and len(layers) > 0:
                if s in d_name or s in slug:
                    for l in layers:
                        l_name = l.get('attributes').get('name').lower()
                        l_id = l.get('id')
                        if app in l.get('attributes').get('application'):
                            if {'d_name': d_name, 'l_name': l_name, 'ds_id': ds_id, 'l_id': l_id} not in found:
                                found.append({'d_name': d_name, 'l_name': l_name, 'ds_id': ds_id, 'l_id': l_id})
                else:
                    for l in layers:
                        l_name = l.get('attributes').get('name').lower()
                        layerSlug = l.get('attributes').get('slug').lower()
                        l_id = l.get('id')
                        if app in l.get('attributes').get('application') and (s in l_name or s in layerSlug):
                            if {'d_name': d_name, 'l_name': l_name, 'ds_id': ds_id, 'l_id': l_id} not in found:
                                found.append({'d_name': d_name, 'l_name': l_name, 'ds_id': ds_id, 'l_id': l_id})

    if len(found) > 0:
        for i, value in enumerate(found):
            print(f"{i} - {value.get('l_name')}\nhttp://api.resourcewatch.org/dataset/{value.get('ds_id')}/layer/{value.get('l_id')}?hash={hash}\n")
    else:
        print('No match! :(')

    return found

In [5]:
def index(found, i):
    hash = random.getrandbits(16)
    ds = found[i]['ds_id']
    l = found[i]['l_id']

    url = f"http://api.resourcewatch.org/dataset/{ds}/layer/{l}?hash={hash}"

    r = requests.get(url)
#     print(r.url)
    layer = r.json().get('data',None)
    updates = layer.get('attributes')
    print(updates.get('name', None))
    
    return {
        'id': {'ds_id': ds, 'l_id': l},
        'app': {'value': updates.get('applicationConfig', None), 'key': 'applicationConfig'},
        'lay': {'value': updates.get('layerConfig', None), 'key': 'layerConfig'},
        'int': {'value': updates.get('interactionConfig', None), 'key': 'interactionConfig'},
        'leg': {'value': updates.get('legendConfig', None), 'key': 'legendConfig'},
        'iso': {'value': updates.get('iso', None), 'key': 'iso'},
        'name': {'value': updates.get('name', None), 'key': 'name'}
    }

In [6]:
def getPayload(key, update, pipeline=False):
    print(f"'{update.get(key).get('key')}': ")
    pprint(update.get(key).get('value'))
    if pipeline:
        return update.get(key).get('value')

In [7]:
def layerUpdate(payload, updates):
    hash = random.getrandbits(16)
    ds = updates.get('id')['ds_id']
    l = updates.get('id')['l_id']
    
    url = f'http://api.resourcewatch.org/dataset/{ds}/layer/{l}'
    print(url)
    headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

    r = requests.patch(url, data=json.dumps(payload), headers=headers)
    pprint(r.json())
    

In [8]:
def searchDataset(search, app):
    search = search.strip().split(' ')
    hash = random.getrandbits(16)
    url = f'https://api.resourcewatch.org/v1/dataset?app={app}&includes=layer,vocabulary,metadata&page[size]=10000&hash={hash}'

    r = requests.get(url)
#     print(r.url)

    datasets = r.json().get('data',None)

    found = []
    for s in search:
        for d in datasets:
            ds_id = d.get('id')
            d_name = d.get('attributes').get('name').lower()
            slug = d.get('attributes').get('slug').lower()

            layers = d.get('attributes').get('layer')

            if layers and len(layers) > 0:
                if s in d_name or s in slug:
                    found.append({'d_name': d_name, 'ds_id': ds_id})
            

    if len(found) > 0:
        for i, value in enumerate(found):
            print(f"{i} - {value.get('d_name')}\nhttp://api.resourcewatch.org/dataset/{value.get('ds_id')}?includes=metadata,vocabulary&hash={hash}\n")
    else:
        print('No match! :(')

    return found

In [9]:
def dsIndex(found, app, i):
    hash = random.getrandbits(16)
    ds = found[i]['ds_id']

    url = f"http://api.resourcewatch.org/dataset/{ds}?includes=metadata,vocabulary&hash={hash}"

    r = requests.get(url)
#     print(r.url)
    dataset = r.json().get('data',None)
    updates = dataset.get('attributes')
    atts = {
        "name": updates.get("name"),
        "subtitle": updates.get("subtitle"),
        "application": updates.get("application"),
        "dataPath": updates.get("dataPath"),
        "attributesPath": updates.get("attributesPath"),
        "connectorType": updates.get("connectorType"),
        "provider": updates.get("provider"),
        "connectorUrl": updates.get("connectorUrl"),
        "tableName": updates.get("tableName"),
        "published": updates.get("published"),
        "env": updates.get("env"),
        "geoInfo": updates.get("geoInfo"),
        "protected": updates.get("protected")
    }
    
    pprint(updates.get('name', None))
    
    metadata = updates.get('metadata', None)
    if metadata:
        for m in metadata:
            if app in m.get('attributes').get('application'):
                meta = m.get('attributes').get('info')
    else:
        meta = {}
    
    vocabulary = updates.get('vocabulary', None)
    vocabs = []
    if vocabulary:
        for v in vocabulary:
            if app in v.get('attributes').get('application'):
                vocabs.append({
                    'tags': v.get('attributes').get('tags'),
                    'name': v.get('attributes').get('name')
                })
    
    return {
        'id': ds,
        'meta': {'value': meta, 'key': 'metadata'},
        'vocab': {'value': vocabs, 'key': 'vocabulary'},
        'att': {'value': atts, 'key': 'attributes'}
    }

In [10]:
def getDsPayload(key, update):
    print(f"'{update.get(key).get('key')}': ")
    pprint(update.get(key).get('value'))
#     return tmp

In [11]:
def dsUpdate(payload, app, update, new=False):
    hash = random.getrandbits(16)
    ds = update['id']
    
    if payload.get('metadata', None):
        
        info = payload.get('metadata')
        url = f'https://api.resourcewatch.org/v1/dataset/{ds}/metadata'
        
        tmp = {
            
            "application": app,
            "language": "en",
            "info": info,
        }
        
        headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}
        
        if new:
            r = requests.post(url, data=json.dumps(tmp), headers=headers)
        elif not new:
            r = requests.patch(url, data=json.dumps(tmp), headers=headers)
            
        print(url)
        pprint(r.json())
        
    elif payload.get('vocabulary', None):
        for v in payload.get('vocabulary'):
            type = v.get('name')
            tags = v.get('tags')
            
            if not new:
                url = f'http://api.resourcewatch.org/dataset/{ds}/vocabulary/{type}?app={app}'
                headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}
                r = requests.delete(url, headers=headers)
                print('Relationship: {type} deleted...')
            
            tmp = {

                "tags": tags,
                "application": app
            }
            
            url = f'https://api.resourcewatch.org/v1/dataset/{ds}/vocabulary/{type}'
            headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}
            r = requests.post(url, data=json.dumps(tmp), headers=headers)
            print(url)
            pprint(r.json())
            
    if payload.get('attributes', None):
        
        payload = payload.get('attributes')
        url = f'http://api.resourcewatch.org/dataset/{ds}'
        print(url)
        headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

        r = requests.patch(url, data=json.dumps(payload), headers=headers)
        pprint(r.json())
        
    

In [12]:
def saveAll(app):
    
    #Get datasets
    hash = random.getrandbits(128)
    url = f'https://api.resourcewatch.org/v1/dataset?app={app}&includes=layer,vocabulary,metadata&page[size]=10000&hash={hash}'

    r = requests.get(url)
    print(r.url)

    datasets = r.json().get('data',None)
    
    today = datetime.datetime.today().strftime('%Y-%m-%d | %Hh %Mm')

    if not os.path.isdir(f"./BACKUP/{today}"):
        os.mkdir(f"./BACKUP/{today}")

    with open(f"./BACKUP/{today}/saved_datasets.json", 'w') as fp:
        json.dump(datasets, fp)

    print("Saved successfully!")


--------------------------------------------

<a class="anchor" id="layers"></a>

# Layer Search

In [163]:
# Search using a key word string e.g. 'Glad', or 'tree cover'.
# Dont worry about caps or spaces!
app = 'gfw'
found = searchLayer('fiber', app)

0 - wood fiber
http://api.resourcewatch.org/dataset/93e67a77-1a31-4d04-a75d-86a4d6e35d54/layer/557dc2cf-0ba7-4410-813c-99d692725fe7?hash=62623

1 - wood fiber 2019
http://api.resourcewatch.org/dataset/93e67a77-1a31-4d04-a75d-86a4d6e35d54/layer/f680828e-be68-4895-b1ed-1d0915d07457?hash=62623



In [168]:
update = index(found, 1)
keys = list(update.keys())[1:]
print(keys)

Wood fiber 2019
['app', 'lay', 'int', 'leg', 'iso', 'name']


In [169]:
getPayload('app', update)

'applicationConfig': 
{'active': True,
 'analysisEndpoint': 'use',
 'default': True,
 'global': True,
 'metadata': 'gfw_wood_fiber'}


In [170]:
# COPYPASTA the above and make changes.  "isImageCollection": false,

payload = {
    
'name': 'Wood fiber',

}


In [171]:
layerUpdate(payload, update)

http://api.resourcewatch.org/dataset/93e67a77-1a31-4d04-a75d-86a4d6e35d54/layer/f680828e-be68-4895-b1ed-1d0915d07457
{'data': {'attributes': {'application': ['gfw'],
                         'applicationConfig': {'active': True,
                                               'analysisEndpoint': 'use',
                                               'default': True,
                                               'global': True,
                                               'metadata': 'gfw_wood_fiber'},
                         'dataset': '93e67a77-1a31-4d04-a75d-86a4d6e35d54',
                         'default': False,
                         'description': 'Wood fiber plantation areas',
                         'env': 'production',
                         'interactionConfig': {'output': [{'column': 'name',
                                                           'format': None,
                                                           'property': 'Name',
                         

--------------------------------------------

<a class="anchor" id="datasets"></a>

# Dataset Search

In [172]:
app = 'gfw'
found = searchDataset('fiber', app)

0 - wood fiber
http://api.resourcewatch.org/dataset/93e67a77-1a31-4d04-a75d-86a4d6e35d54?includes=metadata,vocabulary&hash=63157



In [173]:
update = dsIndex(found, app, 0)
keys = list(update.keys())[1:]
print(keys)

'Wood Fiber'
['meta', 'vocab', 'att']


In [174]:
getDsPayload('att', update)

'attributes': 
{'application': ['gfw'],
 'attributesPath': None,
 'connectorType': 'rest',
 'connectorUrl': 'https://wri-01.carto.com/tables/gfw_wood_fiber',
 'dataPath': None,
 'env': 'production',
 'geoInfo': False,
 'name': 'Wood Fiber',
 'protected': True,
 'provider': 'cartodb',
 'published': True,
 'subtitle': None,
 'tableName': 'gfw_wood_fiber'}


In [175]:
# COPYPASTA here. For vocabs, only change the content of the tags list.
# it will delete the old tag and replace.
# For attributes - dont copy the object parens 

payload = {
    
'attributes': 
{
 'connectorUrl': 'https://wri-01.carto.com/tables/gfw_woodfiber_2019',
 'tableName': 'gfw_woodfiber_2019'
}
    
}

new = False

In [176]:
dsUpdate(payload, app, update, new)

http://api.resourcewatch.org/dataset/93e67a77-1a31-4d04-a75d-86a4d6e35d54
{'data': {'attributes': {'application': ['gfw'],
                         'attributesPath': None,
                         'blockchain': {},
                         'clonedHost': {},
                         'connectorType': 'rest',
                         'connectorUrl': 'https://wri-01.carto.com/tables/gfw_woodfiber_2019',
                         'dataLastUpdated': None,
                         'dataPath': None,
                         'env': 'production',
                         'errorMessage': '',
                         'geoInfo': False,
                         'layerRelevantProps': [],
                         'legend': {'country': [],
                                    'date': [],
                                    'nested': [],
                                    'region': []},
                         'mainDateField': None,
                         'name': 'Wood Fiber',
                        